This notebook requires the "./data/mock_datacube.fits" file. The notebook "simulate_mock_datacube.ipynb" needs to be run to create that datacube.

In [1]:
import numpy as np
from astropy.io import fits

import squirrel
from squirrel.data import Datacube

squirrel.__version__

'0.0.0'

In [2]:
data, header = fits.getdata("./data/mock_datacube.fits", header=True)

In [3]:
data.shape

(362, 35, 35)

In [4]:
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                   35                                                  
NAXIS2  =                   35                                                  
NAXIS3  =                  362                                                  
CRPIX1  =                   17                                                  
CRPIX2  =                   17                                                  
CRVAL1  =                  0.0                                                  
CRVAL2  =                  0.0                                                  
CDELT1  =                  0.1                                                  
CDELT2  =                  0.1                                                  
CTYPE1  = 'RA---TAN'        

In [5]:
wavelengths = header["CRVAL3"] + header["CDELT3"] * (1 + np.arange(header["NAXIS3"]))

datacube = Datacube(data, wavelengths, "um")